In [ ]:
from re import X
# 4. Softmax classification

# Import library
import numpy as np # 수치 연산을 위한 라이브러리
import tensorflow as tf # 머신러닝 프레임워크
import pandas as pd # 데이터 분석을 위한 라이브러리
from sklearn.preprocessing import OneHotEncoder # 카테고리 데이터를 one-hot encoding 하기 위한 라이브러리

# data load
df = pd.read_csv('triangular.csv') #triangular.csv' 파일을 읽어서 데이터프레임 df에 저장



# one-hot encoding
#원-핫 인코딩은 범주형 데이터를 수치형 데이터로 변환하는 인코딩 방식 중 하나임.
#범주형 데이터란, 숫자로 표현되지만 순서나 크기가 의미가 없는 데이터를 의미함. 예를 들어, 성별(남자, 여자), 국적(한국, 미국, 영국) 등이 범주형 데이터에 해당함.
#원-핫 인코딩은 각 범주를 하나의 독립된 차원으로 만들고, 해당 범주에 속하는 데이터에만 1을, 나머지 범주에는 0을 부여하는 방식. 즉, 하나의 범주만 1이고 나머지는 모두 0인 벡터로 표현하는 것.



from sklearn.preprocessing import OneHotEncoder #OneHotEncoder: 객체를 생성하여 클래스 열을 one-hot encoding 함
encoder = OneHotEncoder(sparse_output=False) #클래스를 이용하여 원핫인코더 객채 생성 / sparse를 False로 설정하면 넘파이 배열을 반환하고 True로 설정하면 희소 행렬로 반환한다
encoded_data = encoder.fit_transform(df[['class']]) #fit_transform 메서드를 사용하여 클래스 데이터에 맞게 인코더를 학습하고, one-hot encoding 결과를 encoded_data에 저장
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out()) #one-hot encoding 결과를 데이터프레임 형태로 변환하여 encoded_df에 저장.
#columns=encoder.get_feature_names_out() : 원-핫 인코딩된 컬럼 이름을 설정하는 부분 / 각 열의 이름을 자동으로 생성함


#Pandas DataFrame인 df에서 'class'라는 특정 열만 추출하여 새로운 DataFrame을 생성



df = pd.concat([df, encoded_df], axis=1) #원본 데이터프레임 df에 one-hot encoding된 클래스 열을 추가하기 위해 concat 메서드를 사용

#두 개의 DataFrame을 열(column) 방향으로 합치는 것
#pd.concat: Pandas 라이브러리의 함수로, 여러 개의 DataFrame을 하나로 합치는 역할을 수행
#[df, encoded_df]: 합칠 DataFrame들의 리스트. 여기서는 원본 DataFrame인 df와 원-핫 인코딩된 DataFrame인 encoded_df를 합침
#axis=1: 합치는 방향을 지정합니다.
#axis=0: 행(row) 방향으로 합치는 것으로, DataFrame을 위아래로 쌓는 효과를 냅니다.
#axis=1: 열(column) 방향으로 합치는 것으로, DataFrame을 옆으로 붙이는 효과를 냅니다.



# 10개의 data만 선택
x_data = np.array(df.iloc[1:11,0:3], dtype=np.float32) #데이터프레임에서 1번째부터 10번째 행까지, 처음 3열(0번째부터 2번째)의 데이터를 선택하여 x_data에 NumPy 배열로 저장(특징 데이터).
y_data = np.array(df.iloc[1:11,4:], dtype=np.float32) #데이터프레임에서 1번째부터 10번째 행까지, 4번째 열 이후의 모든 열 (one-hot encoding된 클래스 레이블)을 선택하여 y_data에 NumPy 배열로 저장(레이블 데이터).



#df에서 특정 부분을 추출하여 NumPy 배열 x_data와 y_data로 변환하는 과정
#iloc: DataFrame에서 행과 열을 정수 기반 인덱스로 선택하는 메서드
#1:11: 1번째 행부터 10번째 행까지를 선택. (Python의 인덱싱은 0부터 시작하므로, 11은 포함되지 않.)
#0:3: 0번째 열부터 2번째 열까지를 선택. (3은 포함되지 않음.)



############## Multinomial regression model

# Model parameter
# 모델 파라미터를 작성하세요.
W = tf.Variable(tf.random.normal([3, 3]))
b = tf.Variable(tf.random.normal([3]))

#tf.Variable: TensorFlow에서 변수를 생성하는 함수. 신경망 학습 과정에서 값이 변경되는 가중치와 편향을 저장하기 위해 사용
#tf.random.normal([3, 3]): 3x3 크기의 행렬을 생성하고, 각 원소에 평균이 0이고 표준편차가 1인 정규 분포에서 랜덤하게 값을 할당.
#tf.random.normal([3]): 크기가 3인 벡터를 생성하고, 각 원소에 평균이 0이고 표준편차가 1인 정규 분포에서 랜덤하게 값을 할당

# learning rate
learning_rate = 0.01

# Softmax classifier
def softmax_classifier(): #함수 softmax_classifier를 정의. 이 함수는 소프트맥스 분류기를 구현하고 학습하는 역할.
  with tf.GradientTape() as tape: #TensorFlow의 자동 미분 기능을 활용하기 위해 tf.GradientTape() 컨텍스트 매니저를 사용. 이 블록 내에서 수행되는 연산에 대한 그래프를 기록하여 이후에 기울기를 계산할 수 있음.

    # cost/loss function using softmax classifier
    # cost function을 작성하세요.
    reg = tf.matmul(x_data,W) + b #tf.matmul: 행렬 곱셈을 수행. x_data와 W를 곱한 후 b를 더하여 선형 모델의 출력값인 reg를 계산
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=reg, labels=y_data))
#tf.nn.softmax_cross_entropy_with_logits: 소프트맥스 크로스 엔트로피를 계산하는 함수. logits는 선형 모델의 출력값이고, labels는 실제 레이블.
#tf.reduce_mean: 배치 평균을 계산하여 전체 손실을 구함.

    # Weight update
    # Gradient descent를 구현하세요.
    gradients = tape.gradient(cost, [W,b]) # tape.gradient 함수를 사용하여 손실 함수에 대한 가중치와 편향의 기울기를 계산
    tf.optimizers.SGD(learning_rate).apply_gradients(zip(gradients,[W,b]))
    # tf.optimizers.SGD: 확률적 경사 하강법 옵티마이저
    # learning_rate: 학습률을 설정
    # apply_gradients: 기울기를 사용하여 가중치와 편향을 업데이트하는 함수


############### Learning
for step in range(5001):

  softmax_classifier()

  if step%500 == 0: #500번마다 학습 과정의 결과를 출력

   # Softmax 분류기를 작성하세요.
   model_LC = tf.matmul(x_data,W) + b #선형 모델의 출력값 계산
   model = tf.argmax(tf.nn.softmax(model_LC),1) #소프트맥스 함수를 적용해서 각 클래스에 대한 확률을 계산함 / 가장높은 확률을 가진 클래스를 예측 결과로 저장

   # cost function을 작성하세요.
   cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model_LC, labels=y_data)) #예측값과 실제 레이블 사이의 차이를 계산하여 손실 함수 값을 구함



   # accuracy 계산 /  accuracy, cost 출력
   accuracy = tf.reduce_mean(tf.cast(tf.equal(model, tf.argmax(y_data,1)),tf.float32)) #예측 결과와 실제 레이블이 일치하는 비율을 계산하여 정확도를 구함
   print(step, 'Accuracy : ', accuracy.numpy(), 'Cost : ', cost_function.numpy()) #현재 스텝, 정확도, 손실 함수 값을 출력


         a     b     c            class  class_acute triangle  \
0     3.16  7.72  3.63  obtuse triangle                   0.0   
1     9.46  8.48  7.74   acute triangle                   1.0   
2     7.23  8.58  8.40   acute triangle                   1.0   
3     5.73  3.82  9.95  obtuse triangle                   0.0   
4     3.07  6.10  6.83   right triangle                   0.0   
...    ...   ...   ...              ...                   ...   
1495  3.01  4.86  5.71   right triangle                   0.0   
1496  4.42  9.34  8.23   right triangle                   0.0   
1497  9.24  9.51  6.91   acute triangle                   1.0   
1498  7.86  6.94  3.68   right triangle                   0.0   
1499  6.11  8.91  6.48   right triangle                   0.0   

      class_obtuse triangle  class_right triangle  
0                       1.0                   0.0  
1                       0.0                   0.0  
2                       0.0                   0.0  
3          